In [1]:
# Problem 2: Model Compression
# 1.Compress your new model from problem 1 by converting it into a TF Lite model.
# 2.Use the TensorFlow Interpreter to load your compressed model in (1) and classify new images.
# 3.How big is your compressed model compared to the pre-compressed model in problem 1?
# 4.How quickly does your compressed model classify an image?
# 5.How much did compressing the model affect classification accuracy? 

In [ ]:
#pip install tensorflow_model_optimization # Never used but tried then uninstalled

In [2]:
import os
import tensorflow as tf
import numpy as np
import time
from PIL import Image
import cv2 as cv
import pandas as pd
#import tensorflow_model_optimization as tfmot
#import re
from keras.utils import to_categorical

In [3]:
Train_data = []
Train_labels = []
classes = 43
batch_size = 32
data_directory = './archive'

for i in range(classes):
    path = os.path.join(data_directory, 'Train', str(i))
    images = os.listdir(path)
    for a in images:
        try:
            batch_images = Image.open(os.path.join(path, a))
            batch_images = batch_images.resize((30, 30))
            batch_images = np.array(batch_images, dtype=np.float32)
            Train_data.append(batch_images)
            Train_labels.append(i)
        except:
            print("Error loading image")

Train_data = np.array(Train_data)
#Train_data = Train_data.shuffle(buffer_size=len(Train_data))
Train_labels = np.array(Train_labels)

In [4]:
x_train = tf.convert_to_tensor(Train_data)
y_train = tf.convert_to_tensor(Train_labels)
y_train = to_categorical(y_train, 43)

### Pruning test

In [5]:
"""# Convert the model
import_path = os.path.join(os.getcwd(),'saved_models','model')
pruning = tf.keras.models.load_model(import_path) """

"# Convert the model\nimport_path = os.path.join(os.getcwd(),'saved_models','model')\npruning = tf.keras.models.load_model(import_path) "

In [6]:
"""
# due to downloaded model being a KerasLayers without pruning enabled we couldnt get it to run
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude(pruning)


# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = Train_data.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.00,
                                                               final_sparsity=0.50,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(pruning, **pruning_params)
# prune_low_magnitude = tfmot.sparsity.keras.strip_pruning(pruning)
# re train for 2 epochs?"""

"\n# due to downloaded model being a KerasLayers without pruning enabled we couldnt get it to run\nprune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude(pruning)\n\n\n# Compute end step to finish pruning after 2 epochs.\nbatch_size = 32\nepochs = 2\nvalidation_split = 0.1 # 10% of training set will be used for validation set. \n\nnum_images = Train_data.shape[0] * (1 - validation_split)\nend_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs\n\n# Define model for pruning.\npruning_params = {\n      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.00,\n                                                               final_sparsity=0.50,\n                                                               begin_step=0,\n                                                               end_step=end_step)\n}\n\nmodel_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(pruning, **pruning_params)\n# prune_low_magnitude = tfmot.sparsity.keras.strip_

### Step 1

In [7]:
# Convert the model
import_path = os.path.join(os.getcwd(),'saved_models','model')
converter = tf.lite.TFLiteConverter.from_saved_model(import_path) # path to the SavedModel directory

In [9]:
# tried to implement Post-training integer quantization
# We tried alot of diffrent settings but it seems the model we chose from the tfhub 
# cant be compressed that well not with quantization atleast
# Followed https://www.tensorflow.org/lite/performance/model_optimization 
# and from that https://www.tensorflow.org/lite/performance/post_training_integer_quant
# looked at https://www.tensorflow.org/lite/performance/post_training_quantization for representative dataset
# This reduced Acc of the TfLite model to 3.8% from 45%

"""def representative_data_gen():
  for input_value in Train_data.batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.SELECT_TF_OPS, # Enable TensorFlow ops.
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8 
]
converter.representative_dataset = representative_data_gen"""
# above didnt seem to help the compression despite alot of resourses claiming it would
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.SELECT_TF_OPS, # Enable TensorFlow ops.
]
tflite_model = converter.convert()

# Let's create the folder "saved_tflite_models" and store the TFLite models there
os.makedirs(os.path.join(os.getcwd(),'saved_tflite_models'), exist_ok = True)

# Save the model
tflite_model_path = os.path.join(os.getcwd(),'saved_tflite_models','model.tflite')
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)

### Step 2

In [10]:
test_data = pd.read_csv("./archive/Test.csv")
labels = test_data["ClassId"].values
filenames = test_data["Path"].values

Test_data = []

for filename in filenames:
    img_path = os.path.join("./archive", filename)
    batch_images = Image.open(img_path)
    batch_images = batch_images.resize((30,30))
    batch_images = np.uint8(batch_images)
    Test_data.append(np.array(batch_images))

In [11]:
Test_data =  tf.convert_to_tensor(Test_data)
Test_ds = tf.data.Dataset.from_tensor_slices(Test_data)

AUTOTUNE = tf.data.AUTOTUNE

Test_ds = Test_ds.batch(batch_size).prefetch(AUTOTUNE)

In [12]:
interpreter = tf.lite.Interpreter("./saved_tflite_models/model.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input = input_details[0]['index']
output = output_details[0]['index']

shape_1 = input_shape[1]
shape_2 = input_shape[2]
shape_3 = input_shape[3]

# Resize to allow batches to speed up time for the test dataset
interpreter.resize_tensor_input(input, [batch_size, shape_1, shape_2, shape_3])
interpreter.allocate_tensors()

In [15]:
# Classify all test ds here
pred_probs = []
for batch_images in Test_ds:
  if(batch_images.shape[0]< batch_size): # crashes on last one otherwise (could resize and allocate tensors again but have enough data)
    break
  batch_images = np.float32(batch_images) # not 100% sure this is the right approach
  interpreter.set_tensor(input_details[0]['index'], batch_images)

  interpreter.invoke()
  for pred_out in interpreter.get_tensor(output):
    pred_probs.append(pred_out)

In [16]:
pred_df = pd.DataFrame(np.argmax(pred_probs, axis=1))
pred_df.to_csv("preds_tflite.csv")

### Step 3

In [22]:
def getfoldersize(path):
  total_size = 0
  for dirpath, dirnames, filenames in os.walk(path):
      for f in filenames:
          fp = os.path.join(dirpath, f)
          # skip if it is symbolic link
          if not os.path.islink(fp):
              total_size += os.path.getsize(fp)

  total_size /= (1024 ** 3)
  total_size = round(total_size, 3)
  return total_size

old_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_models','model'))
new_model_size = getfoldersize(os.path.join(os.getcwd(),'saved_tflite_models'))

print("Old GB: " + str(old_model_size))
print("New GB: " + str(new_model_size))

Old GB: 1.468
New GB: 1.446


### Step 4

In [19]:
interpreter = tf.lite.Interpreter("./saved_tflite_models/model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input = input_details[0]['index']
output = output_details[0]['index']

images = cv.imread("./archive/Test/00002.png")
images = cv.cvtColor(images, cv.COLOR_BGR2RGB)
resized_image = cv.resize(images, input_shape[1:3])
input_data = np.expand_dims(resized_image, axis=0)
input_data = np.float32(input_data)

# This is the time per interpreted image
start = time.time()

interpreter.set_tensor(input, input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output)

end = time.time()
print("One image took " + str(round(end - start, 3)) + " sec")

One image took 0.719 sec


### Step 5

In [20]:
# Compere old model and new model acc here (need to load and deload old model)
not_tflite_preds = pd.read_csv("preds.csv")
tflite_preds = pd.read_csv("preds_tflite.csv")

In [21]:
correct = 0
for i, lite in enumerate(tflite_preds.iterrows()):
  if(lite[1].values[-1] == labels[i]):
    correct += 1
print("Lite model Acc: \n" + str(round(correct/len(tflite_preds), 3)))

correct = 0
for i, not_lite in enumerate(not_tflite_preds.iterrows()):
  if(not_lite[1].values[-1] == labels[i]):
    correct += 1

print("Non model Acc: \n" + str(round(correct/len(not_tflite_preds), 3)))

Lite model Acc: 
0.45
Non model Acc: 
0.45
